In [1]:
import numpy as np 
import pandas as pd
import Bio
from Bio.pairwise2 import format_alignment
import copy

import sys 
sys.path.append('../code_general_Finkelsteinlab/')

import Process_SeqLibrary_Finkelsteinlab as preprocess

import plotting_Finkelsteinlab as pltData

%matplotlib inline 
import matplotlib.pylab as plt 

In [2]:
import Prepare_data as Pre
path = '../Data_ABA_Finkelsteinlab/' 
filename= 'cas9-target-e-replicate-1-delta-abas_Canonical_OT-r_0-2.csv'
xdata,ydata,yerr=Pre.Prepare_Cdata(path,filename)
print(xdata[0:2])
print(ydata[0:2])
print(yerr[0:2])

[array([1]), array([10])]
[[2.107582, 2.2418720000000003, 2.19105, 2.128268, 2.136484, 2.15305, 2.2701990000000003, 2.211512, 2.178649, 2.1892560000000003, 2.210899, 2.231373, 2.142187, 1.848123, 1.97554, 2.183333, 2.049935, 2.199445, 2.240856, 2.242004, 2.201306, 2.184312, 2.202237, 2.271958, 2.196158, 2.156701, 2.130128, 2.1343900000000002, 2.153825, 2.150444, 1.962753, 2.235647, 2.200695, 2.069164, 2.1381330000000003, 2.155552, 2.179248, 2.083879, 1.8498139999999998, 2.1984169999999996, 2.1897729999999997, 2.2370900000000002, 2.180273, 2.185925, 2.413176, 2.192801, 2.079257, 1.967932], [0.212308, 0.62721, 0.5690470000000001, 0.7840779999999999, 0.327397, 0.795272, 0.46398999999999996, 0.853942, 0.533552, 0.794582, 0.47755200000000003, 0.9220379999999999, 0.891862, 0.667221, 0.363523, 0.554671, 0.461413, 0.789131, 0.7857029999999999, 0.681701, 0.44807700000000006, 0.7963680000000001, 0.798054, 0.8555229999999999, 0.307153, 0.883461, 0.315629, 0.49782, 0.726675, 0.8466049999999999, 0.

In [3]:
upper_bnd =  [10.0] + [10.0]*40 +  [3.0] *2
lower_bnd = [0.0] + [-10.0]*40 + [-7.0] *2
initial_guess =  [5.0] + [0.0]*40 + [0.0] *2

In [4]:
import Chisq_Finkelstein as Chi
import SimulatedAnnealing_Finkelstein_parallel as SA
import sys
import Calculate_ABA_Finkelsteinlab_Diewertje as ABA

import functools

In [6]:
# parameters =  np.array([5.0] + [1.0]*40 + [0.5] *2)
# concentrations = np.array([0.1, 0.3, 1, 3, 10, 30, 100, 300]) # in nanoMolair
# reference=1 # in nanomolair

# ontarget_ABA=ABA.calc_ABA(parameters,concentrations,reference, mismatch_positions=[],model_id = 'general_energies', guide_length = 20, T=10*60)
# Pbound=ABA.calc_Pbound(parameters,concentrations,reference,mismatch_positions=[],model_id = 'general_energies', guide_length = 20, T=10*60)
# print(Pbound)

#mABA=ABA.calc_delta_ABA(parameters,concentrations,reference, xdata[10],ontarget_ABA,model_id = 'general_energies', guide_length = 20, T=10*60)
#print(mABA)
#chi=Chi.calc_Chi_square(parameters,xdata[10],np.ones(len(yerr[10]))*mABA,yerr[10],concentrations,
        #                     reference,guide_length=20,model_id='general_energies')
#print(chi)
# THIS WAS TO CHECK IF CHI IS 0 IF THE mABA IS THE SAME AS YDATA

#chi=Chi.calc_Chi_square(parameters,xdata[1],ydata[1],yerr[1],concentrations,
 #                            reference,guide_length=20,model_id='general_energies')
#print(chi)

ABA.get_master_equation(parameters,mismatch_positions=[],model_id='general_energies', guide_length=20)
# plot to see the fit
# import matplotlib.pylab as plt
# plt.plot(range(10),mABA)
# for i in range(10):
#     m=np.ones(len(ydata[i]))*i
#     plt.scatter(m,ydata[i])
# #KineticModel(parameters,xdata[0],ydata[0],yerr[0])
# print(mABA)
# new_ydata=mABA

# IT DOES RUN, SO THE CHI SQUARE FUNCTION WORKS, THE REASON WHY IT IS SUCH A POOR FIT,
# IS THAT I JUST TOOK SOME INITIAL PARAMETERS

# Om het te laten werken, remove the # in the functie in python file!

____________
[5.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
 1.  1.  1.  1.  1.  0.5 0.5]
_________________
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0. ]


array([[ -0.5       ,  74.20657955,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.5       , -74.70657955,   0.18393972,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.5       ,  -0.68393972,   0.18393972,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.    

In [ ]:
gRNA_length = 20
model_ID = 'general_energies'
parameters =  np.array([5.0] + [1.0]*40 + [0.5] *2)
concentrations = np.array([0, 0.1, 0.3, 1, 3, 10, 30, 100, 300]) # in nanoMolair
reference=1 # in nanomolair

KineticModel = functools.partial(Chi.calc_Chi_square,model_id=model_ID, guide_length=20,
                                 concentrations=concentrations,reference=reference)
chi=KineticModel(parameters,xdata[0:10],ydata[0:10],yerr[0:10])
print(chi)

In [ ]:
new_ydata=mABA
chi,mABA=Chi.calc_Chi_square(parameters,xdata[0:10],new_ydata,yerr[0:10],concentrations,
                             reference,guide_length=20,model_id='general_energies')
print(chi)
# plot to see the fit
import matplotlib.pylab as plt
plt.plot(range(10),mABA)
for i in range(10):
    m=np.ones(len(ydata[i]))*i
    plt.scatter(m,ydata[i])

In [ ]:
# to try without the working function from berouz
df=pd.DataFrame()
df['MMpos']=['1|6','1|6','7|2','8|3','19|1','15,8']
df['ABA']=[6,4,9,3,8,2]
df['Uncertainty']=[0.1,0.03,0.3,0.2,0.68,0.9]
Grouped = df.groupby('MMpos').agg(lambda x: list(x))
Grouped.reset_index(inplace=True)
Grouped


In [ ]:
s='7|22'
s.split('|')
mm = np.array(s.split('|')).astype(int)
mm

In [ ]:
MMpos=Grouped['MMpos'].tolist()
ABA=Grouped['ABA'].tolist()
Uncertainty=Grouped['Uncertainty'].tolist()

#df['df.apply('functionname',axis=1)

In [ ]:
Path = '../Data_ABA_Finkelsteinlab/cas9-target-e-replicate-1-delta-abas.txt'
data = pd.read_table(Path,comment='#')
#data

In [ ]:
Cas='Cas9'
on_target = 'TTTAGACGCATAAAGATGAGACGCTGG'
seq_colname = 'Sequence'
output_colnames = ['Delta ABA (kBT)', 'Uncertainty']
out_file_name = 'Cas9_target_e.csv'
out_path = '../Data_ABA_Finkelsteinlab/cas9-target-e-replicate-1-delta-abas.txt'

new_data = preprocess.process_Finkelstein_Library(data, on_target, seq_colname, output_colnames, Cas, 
                                                  Canonical_PAM = True, Mut_type = ['OT','r'], Mut_min = 0, Mut_max = 20,
                                                  out_file_name = out_file_name, out_path = out_path, nofilter = False,
                                                  save_data = True)


In [ ]:
fit_result_file= 'fit_results.txt'
monitor_file = 'monitor.txt'
init_monitor_file='init_monitor.txt'

gRNA_length = 20
model_ID = 'general_energies'
parameters =  np.array([5.0] + [1.0]*40 + [0.5] *2)
concentrations = np.array([0.1, 0.3, 1, 3, 10, 30, 100, 300]) # in nanoMolair
reference=1 # in nanomolair

KineticModel = functools.partial(Chi.calc_Chi_square,model_id=model_ID, guide_length=20,
                                 concentrations=concentrations,reference=reference)

fit_result = SA.sim_anneal_fit(xdata=xdata[0],
                                ydata=ydata[0],
                                yerr = yerr[0],
                                Xstart= np.array(initial_guess),
                                lwrbnd= np.array(lower_bnd),
                                upbnd= np.array(upper_bnd),
                                model='I_am_using_multi_processing_in_stead',
                                objective_function=KineticModel,
                                on_target_function=ABA.calc_ABA,
                                Tstart=100.,             # infered from run on my computer/other runs on cluster
                                use_relative_steps=False,
                                delta=1.0,
                                tol=1E-5,
                                Tfinal=50,
                                adjust_factor=1.1,
                                cooling_rate=0.5,
                                N_int=10,
                                AR_low=0,
                                AR_high=100,
                                use_multiprocessing=True,
                                nprocs=1,
                                output_file_results = fit_result_file,
                                output_file_monitor = monitor_file,
                                output_file_init_monitor=init_monitor_file
                              )

In [ ]:
def f(a,b):
    return a+b
import functools
g=functools.partial(f,b=5)
print(f(1,5))
print(g(1))

In [9]:
len(np.loadtxt('parameters.txt'))

44